In [1]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks
%ls
# pour une utilisation sur une machine locale changer le chemin ci-dessous et décommenter la ligne
#%cd ./

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks
 1_-_Classification_de_donnees.ipynb
 3_ClassDonneeText/
 3_-_Classification_de_donnees_textuelles.ipynb
 6_-_Les_word_embeddings_solutionsintegrees.ipynb
 amazon_cells_labelled.txt
 LSTMForGeneratingText.ipynb
 LSTMForGeneratingText_VersioCommente.ipynb
 LSTMForGeneratingText-VersioCommente.ipynb
 SPOILER_4_-_Utiliser_un_modele_solutionsintegrees.ipynb
 Untitled
'Untitled (1)'
'Untitled (2)'
'Untitled (3)'
 weights-improvement-50-1.1972-bigger.hdf5
 wonderland.txt


In [55]:
#https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pandas as pd
# load ascii text and covert to lowercase

#charge fichier
# filename = "ReviewsLabelled.csv"
# raw_text = open(filename, 'r', encoding='utf-8').read()

df = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')

object_dataframe = df[df['sentiment'] >= 1].sentence
raw_text = str(object_dataframe)
### pre-traitement du texte ###

raw_text = raw_text.lower() #texte en minuscule
chars = sorted(list(set(raw_text))) # creation dictionnaire
char_to_int = dict((c, i) for i, c in enumerate(chars)) # code du dico en entiers

n_chars = len(raw_text) #nb caractere du texte
n_vocab = len(chars) #nb de lettres distincte

# print ("Tous les characteres: ",chars)
# print ("Total Characters: ", n_chars)
# print ("Total Vocab: ", n_vocab)

### Creation des matrices entré / sortie ###
seq_length = 100 #100 pour Alice
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  # print (n_chars - seq_length,' ',i,"raw_text[i:i + seq_length]",raw_text[i:i + seq_length],' raw_text[i + seq_length]',raw_text[i + seq_length],char_to_int[raw_text[i + seq_length]])
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
  
n_patterns = len(dataX) #nombre de patterns / taille de X --> 34
X = numpy.reshape(dataX, (n_patterns, seq_length, 1)) # on change X tableau --> matrice (34 x 10 x 1)

X = X / float(n_vocab) #normaliser -> 0-1
y = np_utils.to_categorical(dataY) # tableau 34 --> matrice (34 x 17) de O et 1, exemple : lettre a predire 6 --> 1 en 6eme pos

### define the LSTM model ###
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5" #fichier pour sauvegarde
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min') #on sauvegarde
callbacks_list = [checkpoint] #liste des sauvegardes

### entrainement ###
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
10/10 [==============================] - 4s 74ms/step - loss: 3.3888

Epoch 00001: loss improved from inf to 3.11801, saving model to weights-improvement-01-3.1180-bigger.hdf5
Epoch 2/50
10/10 [==============================] - 1s 61ms/step - loss: 2.6688

Epoch 00002: loss improved from 3.11801 to 2.71368, saving model to weights-improvement-02-2.7137-bigger.hdf5
Epoch 3/50
10/10 [==============================] - 1s 61ms/step - loss: 2.6920

Epoch 00003: loss improved from 2.71368 to 2.66782, saving model to weights-improvement-03-2.6678-bigger.hdf5
Epoch 4/50
10/10 [==============================] - 1s 60ms/step - loss: 2.6636

Epoch 00004: loss improved from 2.66782 to 2.61061, saving model to weights-improvement-04-2.6106-bigger.hdf5
Epoch 5/50
10/10 [==============================] - 1s 60ms/step - loss: 2.5986

Epoch 00005: loss improved from 2.61061 to 2.59028, saving model to weights-improvement-05-2.5903-bigger.hdf5
Epoch 6/50
10/10 [==============================]

In [69]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

df = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')
object_dataframe = df[df['sentiment'] >= 1].sentence
raw_text = str(object_dataframe)

# print(raw_text)

raw_text = raw_text.lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

filename = "weights-improvement-47-1.9832-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# start = numpy.random.randint(0, len(dataX)-1) #on creer un seed un nb alea entre O et le nb_char
# pattern = dataX[start] #les 100 caracteres apres la position start
# print ("Seed:")
# print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# print(pattern)

seedTexte = "i bought this to use with my kindle fire and absolutely loved it. this product is ideal for people a"
pattern = [char_to_int[value] for value in seedTexte]
# print(pattern)


# generate characters
for i in range(10):
  x = numpy.reshape(pattern, (1, len(pattern), 1)) #on reshape l'entrée
  x = x / float(n_vocab)#normalise
  prediction = model.predict(x, verbose=0) #predire le caractere suivant

  index = numpy.argmax(prediction) #index du caractere avec la plus grande proba de prediction
  result = int_to_char[index] #entier --> char
  sys.stdout.write(result)

  seq_in = [int_to_char[value] for value in pattern]#pattern en entiers
  pattern.append(index) #on ajoute le nouveau caractere
  pattern = pattern[1:len(pattern)]# on eneleve le premier


!!!!!!!!!.

In [58]:
# librairies gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors # to load gzip bin files with KeyedVectors.load_word2vec_format
from gensim import models
from gensim.models.word2vec import Word2Vec
import numpy as np

from gensim import models
model_stanford = models.KeyedVectors.load_word2vec_format("glove.6B.100d.w2vformat.txt", binary=False)


FileNotFoundError: ignored

In [ ]:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

def get_embedding_matrix(embeddings,words,vocabulary_size,dimension):
  # Preparation de la matrice des embeddings embedding matrix
  embedding_matrix = np.zeros((vocabulary_size, dimension)) #(22 x 100)
  for word, i in words.word_index.items():
    try: # lever l'exception si un mot n'est pas trouvé dans les embeddings
        embedding_vector = embeddings.wv[word] 
        embedding_matrix[i] = embedding_vector
    except KeyError: # mot non trouvé dans les embeddings
        # print(word)
        pass
  return embedding_matrix


# source text
filename = "wonderland.txt"
data = open(filename, 'r', encoding='utf-8').read()
# data = """ Jack and Jill went up the hill\n
# 		To fetch a pail of water\n
# 		Jack fell down and broke his crown\n
# 		And Jill came tumbling after\n """

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0] #codage des mots

vocab_size = len(tokenizer.word_index) + 1 #pourquoi +1 ???
print('Vocabulary Size: %d' % vocab_size)

# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)

# pad sequences
max_length = max([len(seq) for seq in sequences]) #taille de la sequence la plus longue ? 3
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') #padding

# split into input and output elements
sequences = array(sequences) #utile??
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

DIMENSION=100 #max_length#20
embedding_matrix=get_embedding_matrix(model_stanford,tokenizer,vocab_size,DIMENSION)

#---------- model ---------------
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length-1))#, trainable=False)) 
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print(model.summary())
model.fit(X, y, epochs=100, verbose=2)
model.save('model_embeddings.bin')

Vocabulary Size: 2862


NameError: ignored

In [ ]:
from tensorflow import keras

new_model = keras.models.load_model('model_embeddings.bin')

print(generate_seq(new_model, tokenizer, max_length-1, 'so long', 10))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


so long since she had not gone much farther before she had
